In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv('news.csv')

c:\users\t george\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

Unnamed: 0                                              title  \
0       8476                       You Can Smell Hillary's Fear   
1      10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2       3608        Kerry to go to Paris in gesture of sympathy   
3      10142  Bernie supporters on Twitter erupt in anger ag...   
4        875   The Battle of New York: Why This Primary Matters   

                                                text label Unnamed: 4  \
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE        NaN   
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE        NaN   
2  U.S. Secretary of State John F. Kerry said Mon...  REAL        NaN   
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE        NaN   
4  It's primary day in New York and front-runners...  REAL        NaN   

  Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 131  \
0        NaN        NaN        NaN        NaN        NaN  ...          NaN   
1        NaN        NaN        NaN        NaN        NaN  ...          NaN   
2        NaN        NaN        NaN        NaN        NaN  ...          NaN   
3        NaN        NaN        NaN        NaN        NaN  ...          NaN   
4        NaN        NaN        NaN        NaN        NaN  ...          NaN   

  Unnamed: 132 Unnamed: 133 Unnamed: 134 Unnamed: 135 Unnamed: 136  \
0          NaN          NaN          NaN          NaN          NaN   
1          NaN          NaN          NaN          NaN          NaN   
2          NaN          NaN          NaN          NaN          NaN   
3          NaN          NaN          NaN          NaN          NaN   
4          NaN          NaN          NaN          NaN          NaN   

  Unnamed: 137 Unnamed: 138 Unnamed: 139 Unnamed: 140  
0          NaN          NaN          NaN          NaN  
1          NaN          NaN          NaN          NaN  
2          NaN          NaN          NaN          NaN  
3          NaN          NaN          NaN          NaN  
4          NaN          NaN          NaN          NaN  

[5 rows x 141 columns]

In [5]:
dfreal = df.loc[:,:'label']

In [6]:
dfreal.head()

Unnamed: 0                                              title  \
0       8476                       You Can Smell Hillary's Fear   
1      10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2       3608        Kerry to go to Paris in gesture of sympathy   
3      10142  Bernie supporters on Twitter erupt in anger ag...   
4        875   The Battle of New York: Why This Primary Matters   

                                                text label  
0  Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1  Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2  U.S. Secretary of State John F. Kerry said Mon...  REAL  
3  — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4  It's primary day in New York and front-runners...  REAL

In [7]:
dfreal.set_index('Unnamed: 0')

title  \
Unnamed: 0                                                      
8476                             You Can Smell Hillary's Fear   
10294       Watch The Exact Moment Paul Ryan Committed Pol...   
3608              Kerry to go to Paris in gesture of sympathy   
10142       Bernie supporters on Twitter erupt in anger ag...   
875          The Battle of New York: Why This Primary Matters   
...                                                       ...   
4490        State Department says it can't find emails fro...   
8062        The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
8622        Anti-Trump Protesters Are Tools of the Oligarc...   
4021        In Ethiopia, Obama seeks progress on peace, se...   
4330        Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                         text label  
Unnamed: 0                                                           
8476        Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
10294       Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
3608        U.S. Secretary of State John F. Kerry said Mon...  REAL  
10142       — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
875         It's primary day in New York and front-runners...  REAL  
...                                                       ...   ...  
4490        The State Department told the Republican Natio...  REAL  
8062        The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...  FAKE  
8622         Anti-Trump Protesters Are Tools of the Oligar...  FAKE  
4021        ADDIS ABABA, Ethiopia —President Obama convene...  REAL  
4330        Jeb Bush Is Suddenly Attacking Trump. Here's W...  REAL  

[7795 rows x 3 columns]

In [8]:
dfreal.shape

(7795, 4)

In [9]:
y = dfreal.label

In [10]:
df = dfreal.set_index('Unnamed: 0')

In [11]:
df = df.drop(['label'], axis=1)

In [12]:
df['text'].fillna('no text', inplace=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size = 0.2, random_state= 53)

In [14]:
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.astype(str))
count_test = count_vectorizer.transform(X_test.astype(str))

In [15]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train.astype(str))
tfidf_test = tfidf_vectorizer.transform(X_test.astype(str))

In [16]:
tfidf_vectorizer.get_feature_names()[-10:]

['שתי',
 'תאמצנה',
 'תוצאה',
 'תחל',
 'תיירות',
 'תנותק',
 'תעודת',
 'תתרכז',
 'عربي',
 'ยงade']

In [17]:
count_vectorizer.get_feature_names()[:10]

['00',
 '000',
 '0000',
 '000000031',
 '00000031',
 '000035',
 '00006',
 '0001',
 '0002',
 '000billion']

In [18]:
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

In [19]:
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())

MemoryError: Unable to allocate 2.82 GiB for an array with shape (6236, 60655) and data type float64

In [ ]:
difference = set(count_df.columns) - set(tfidf_df.columns)
difference

In [ ]:
print(count_df.equals(tfidf_df))

In [ ]:
count_df.head()

In [ ]:
tfidf_df.head()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
clf = MultinomialNB()

In [ ]:
y_train.fillna('FAKE',inplace=True)

In [ ]:
y_test.fillna('FAKE',inplace=True)

In [ ]:
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred, labels=['FAKE', 'REAL'])
plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])

In [ ]:
clf = MultinomialNB()

In [ ]:
clf.fit(count_train, y_train)
pred = clf.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
cm = metrics.confusion_matrix(y_test, pred, labels=['FAKE', 'REAL'])
plot_confusion_matrix(cm, classes=['FAKE', 'REAL'])